In [17]:
import sys
print("Python Version: ",sys.version)

Python Version:  3.11.11 (main, Dec  4 2024, 08:55:07) [GCC 11.4.0]


# CUDA 설정
로컬의 경우 cuda와 별도의 라이브러리 설치 필요

In [18]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [19]:
!pip install transformers accelerate huggingface_hub safetensors tqdm ipywidgets tiktoken blobfile protobuf sentencepiece

# HuggingFace Sign-in

In [20]:
from huggingface_hub import notebook_login

notebook_login()

# Dataset

In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [22]:
# 입력 텍스트
precondition = """
오늘은 2025년 1월 14일입니다.
일정:
- 2025년 1월 11일 오전 10시: 회의
- 2025년 1월 15일 오후 3시: 병원 방문
"""
utterance = "내일의 일정을 알려주세요. 없으면 '일정 없음'이라고 답해주세요."
system_message="당신은 일정 관리 비서입니다. 사용자의 질문에 정확히 답하세요."

input_message = f"{precondition}\n질문: {utterance}"

dialogue_messages = [
    {"role": "system", "content": system_message },
    {"role": "user", "content": input_message}
]

print(input_message)


오늘은 2025년 1월 14일입니다.
일정:
- 2025년 1월 11일 오전 10시: 회의
- 2025년 1월 15일 오후 3시: 병원 방문

질문: 내일의 일정을 알려주세요. 없으면 '일정 없음'이라고 답해주세요.


# LLama-7b

In [23]:
model_name = "meta-llama/Llama-2-7b-hf"

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)

# 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",  # GPU 활용 시 자동 설정
    device_map="auto",   # 모델을 GPU로 로드
    use_auth_token=True  # Hugging Face 토큰 인증
)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model-00001-of-00002.safetensors:   5%|5         | 514M/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [25]:
# 입력 텍스트

inputs = tokenizer(input_message, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=100,        # 최대 출력 길이 제한
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


오늘은 2025년 1월 14일입니다.
일정:
- 2025년 1월 11일 오전 10시: 회의
- 2025년 1월 15일 오후 3시: 병원 방문

질문: 내일의 일정을 알려주세요. 없으면 '일정 없음'이라고 답해주세요.

답: 오늘은 2025년 1월 14일입니다.
일정:
- 2025년 1월 11일 오전 10시: 회의
- 2025년 1월 15일 오후 3시: 병원 방문

질문:


# phi-4

In [26]:
model_name = "microsoft/phi-4"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto",
    use_auth_token=True
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [27]:
from transformers import pipeline

chat_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

response = chat_pipeline(dialogue_messages, max_new_tokens=128)
print(response[0]['generated_text'][2]['content'])

Device set to use cuda:0


내일은 2025년 1월 15일입니다. 당신의 일정에는 다음과 같은 항목이 있습니다:

- 오후 3시: 병원 방문

이 외에는 추가 일정이 없습니다.
